In [5]:
using CSV, FileIO, DataFrames, Dates
using GMT

In [6]:
region="vrancea"
mkpath("./results/$region/")

df = CSV.read("./data/$region.csv", DataFrame);
first(df,5)

Row,Datetime,Latitude,Longitude,Depth,Magnitude
,DateTime,Float64,Float64,Float64,Float64
1,1977-03-04T19:21:54.100,45.77,26.76,94.0,7.4
2,1977-03-04T21:21:01.100,45.22,26.65,141.0,3.0
3,1977-03-05T02:35:22,45.63,26.19,121.3,3.0
4,1977-03-05T12:08:42,45.37,26.3,124.0,3.5
5,1977-03-06T13:27:06.300,45.83,26.63,112.3,2.6


In [7]:
min_lon = minimum(df.Longitude)
max_lon = maximum(df.Longitude)
min_lat = minimum(df.Latitude)
max_lat = maximum(df.Latitude)
min_dep = minimum(df.Depth)
max_dep = maximum(df.Depth);

In [8]:
map_coords = (min_lon,max_lon,min_lat,max_lat)
map_coords_depth = (min_lon,max_lon,min_lat,max_lat,-max_dep,min_dep)

(26.0, 27.0, 45.2, 46.0, -196.0, 50.0)

In [9]:
coast(region=map_coords, proj=:merc, land=:gray, rivers="a", shore="0.25p", show=true)

In [10]:
function scatter_2D(df, region, magnitude_threshold; z_control="Magnitude")

    df = df[df.Magnitude .>= magnitude_threshold,:];

    # Get region's coordinates
    min_lon = minimum(df.Longitude)
    max_lon = maximum(df.Longitude)
    min_lat = minimum(df.Latitude)
    max_lat = maximum(df.Latitude);

    # Create the map coordinates
    map_coords = (min_lon,max_lon,min_lat,max_lat)
    
    # Colormap for the region topography
    C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
    # Relief map of the region
    relief_map = grdcut("@earth_relief_30s", region=map_coords);

    
    # control marker size based on magnitude
    marker_size = [2^x/100 for x in df.Magnitude];

    # control marker color either by Magnitude or by Depth
    C_markers = makecpt(cmap=:seis, range=(minimum(df[!, z_control]),maximum(df[!, z_control])),continuous=true, inverse=true);
    colorbar_label = z_control
    zcolor_control = df[!, z_control]

    basemap(region=map_coords,frame=(axes=:WSne), proj=:merc)

    grdview!(relief_map, proj=:merc, axis=:none, surftype=(image=1000,), 
            cmap=C_map, zsize=1.5, alpha=40)

    plot!(df.Longitude, df.Latitude, 
            markersize=marker_size, marker=:cc, markerline=:faint,
            cmap=C_markers, zcolor=zcolor_control, alpha=60)

    colorbar!(pos=(outside=:MR, offset=(1.0,0)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel=colorbar_label,),par=(MAP_LABEL_OFFSET=0.8,), 
                savefig="./results/$region/$(region)_2D_mag_$(magnitude_threshold)_$(colorbar_label).png", show=true)

end

scatter_2D (generic function with 1 method)

In [ ]:
function scatter_semi_3D(df, region, magnitude_threshold; z_control="Magnitude")

    df = df[df.Magnitude .>= magnitude_threshold,:];

    # Get region's coordinates
    min_lon = minimum(df.Longitude)
    max_lon = maximum(df.Longitude)
    min_lat = minimum(df.Latitude)
    max_lat = maximum(df.Latitude);

    # Create the map coordinates
    map_coords = (min_lon,max_lon,min_lat,max_lat)
    
    # Colormap for the region topography
    C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
    # Relief map of the region
    relief_map = grdcut("@earth_relief_30s", region=map_coords);

    
    # control marker size based on magnitude
    marker_size = [2^x/100 for x in df.Magnitude];

    # control marker color either by Magnitude or by Depth
    C_markers = makecpt(cmap=:seis, range=(minimum(df[!, z_control]),maximum(df[!, z_control])),continuous=true, inverse=true);
    zcolor_control = df[!, z_control]


    basemap(region=map_coords,frame=(axes=:SE), proj=:merc, view=(145,45))

    grdview!(relief_map, proj=:merc, axis=:none, surftype=(image=1000,), 
            cmap=C_map, zsize=1.5, alpha=40 , view=(145,45))

    plot!(df.Longitude, df.Latitude, 
            markersize=marker_size, marker=:cc, markerline=:faint,
            cmap=C_markers, zcolor=zcolor_control, alpha=60, view=(145,45))

    colorbar!(pos=(outside=:MR, offset=(1.5,0)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel=z_control,),par=(MAP_LABEL_OFFSET=0.8,), 
                view=(145,45), savefig="./results/$region/$(region)_semi3D_mag_$(magnitude_threshold)_$(z_control).png", show=true)

end

In [11]:
scatter_2D(df,region,2.0,z_control="Magnitude")

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
